In [ ]:
import pypsa
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns
from datetime import datetime
from cartopy import crs as ccrs
from pypsa.plot import add_legend_circles, add_legend_lines, add_legend_patches
import plotly.express as px
import hvplot.pandas
import matplotlib.colors as mcolors

In [ ]:
country = "NZ"

In [ ]:
# Read ports geodata from csv file with geopandas and use x and y coordinates to plot
fn="../../resources/pypsa-earth-sec/data/ports.csv"
fn_export="../../resources/pypsa-earth-sec/data/export_ports.csv"
OUTPUT = "../../results/figures/ports_exportports.pdf"

In [ ]:
export_ports = pd.read_csv(fn_export)
geometry = gpd.points_from_xy(export_ports.x, export_ports.y)
export_ports = gpd.GeoDataFrame(export_ports, geometry=geometry)

export_ports = export_ports[export_ports['country'] == country]

# Set crs to WGS84
export_ports = export_ports.set_crs(epsg=4326)

# Convert fraction to float
export_ports['fraction'] = export_ports['fraction'].astype(float)

In [ ]:
ports = gpd.read_file(fn)
geometry = gpd.points_from_xy(ports.x, ports.y)
ports = gpd.GeoDataFrame(ports, geometry=geometry)

# Filter ports to only include a country 
ports = ports[ports['country'] == country]

# Set crs to WGS84
ports = ports.set_crs(epsg=4326)

# Convert fraction to float
ports['fraction'] = ports['fraction'].astype(float)


### Static plot

In [ ]:
regions_onshore = gpd.read_file("../../results/shapes/country_shapes.geojson")
n=pypsa.Network("../../results/networks/elec.nc")

In [ ]:
capacity_scale = 5e-3
extent = [165.0, 180.0, -48.0, -33.0] # for country NZ [165.0, 180.0, -50.0, -25.0] 

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection": ccrs.PlateCarree(), "frameon": False})

regions_onshore.plot(
    ax=ax,
    facecolor="whitesmoke",
    edgecolor="black",
    aspect="equal",
    transform=ccrs.PlateCarree(),
    linewidth=0.2,
)

ports.plot(
    ax=ax,
    markersize= ports["Harbor_size_nr"].values.astype(float) / capacity_scale,
    alpha=0.7,
    legend=True,
    legend_kwds=dict(frameon=True, facecolor="white", loc="lower right"), #title="Technology (size ~ capacity)", 
    transform=ccrs.PlateCarree()
)

export_ports.plot(
    ax=ax,
    markersize= ports["Harbor_size_nr"].values.astype(float) / capacity_scale,
    alpha=0.7,
    legend=True,
    color="yellow",
    legend_kwds=dict(frameon=True, facecolor="white", loc="lower right"), #title="Technology (size ~ capacity)", 
    transform=ccrs.PlateCarree()
)


ax.set_extent(extent)
fig.tight_layout()
fig.savefig(OUTPUT, bbox_inches="tight", dpi=300)

### Interactive

In [ ]:
# Create a scatter map with Mapbox as the basemap
fig = px.scatter_mapbox(ports, 
        lat=ports.geometry.y, 
        lon=ports.geometry.x, 
        mapbox_style="carto-positron",
        zoom=5, 
        center=dict(lat=31.7917, lon=-7.0926), 
        height=600, 
        width=800,
        size='fraction',
        hover_name='name',
        )
fig.show()

# Save the figure
#fig.to_image(format="png", engine="orca")